# PDF2TXT
### CONFIGURAÇÃO DO AMBIENTE
Verifica se as bibliotecas neecessárias estão instaladas.

In [6]:
!pip install pdfplumber ipywidgets ipyfilechooser


## IMPORTAÇÃO DAS BIBLIOTECAS
pdfplumber para a extração de texto
ipywidgets para o upload interativo de arquivos.

In [22]:
import base64
from io import BytesIO
import pdfplumber
import time
import os
from ipywidgets import FileUpload, Button, Output,VBox, HTML
from IPython.display import display, FileLink, HTML, clear_output
from ipyfilechooser import FileChooser


## Widget UPLOADER
Permite que o usuário faça o upload de múltiplos arquivos PDF para extração de texto;

In [30]:
# Célula 3: Widget para upload do arquivo PDF
uploader = FileUpload(accept='.pdf', multiple=True)
display(uploader)

FileUpload(value={}, accept='.pdf', description='Upload', multiple=True)

## EXTRAÇÃO DO TEXTO
O código aguarda o upload do arquivo.
Para cada PDF enviado, ele abre o arquivo e itera por cada página, extraindo o texto disponível.
O texto extraído é disponibiilizado para download.
Automaticamente, o texto é salvo em um arquivo .txt cujo nome é derivado do nome original do PDF.
Extrai o texto de cada página (ignorando as imagens).
Cria um conteúdo TXT cujo nome é baseado no nome do PDF.

In [29]:
# Aguarda até que pelo menos um arquivo seja carregado
while not uploader.value:
    time.sleep(0.5)

# Lista para armazenar os widgets de download para cada arquivo
botoes_download = []

# Processa cada arquivo PDF enviado
for filename, file_info in uploader.value.items():
    print(f"Processando: {filename}")
    # Cria um objeto BytesIO a partir do conteúdo do arquivo
    pdf_bytes = BytesIO(file_info['content'])

    # Abre o PDF e extrai o texto
    with pdfplumber.open(pdf_bytes) as pdf:
        texto_extraido = ""
        for i, page in enumerate(pdf.pages):
            page_text = page.extract_text()
            if page_text:
                texto_extraido += f"\n--- Página {i+1} ---\n{page_text}"
            else:
                texto_extraido += f"\n--- Página {i+1} não contém texto extraível. ---\n"

    # Define o nome do arquivo TXT (ex.: documento_extraido.txt)
    nome_txt = filename.replace('.pdf', '_extraido.txt')

    # Função para criar um "botão" de download (link HTML com aparência de botão)
    def criar_botao_download(texto, nome_arquivo):
        """
        Cria um HTML que se apresenta como um botão para download do conteúdo passado.
        """
        # Codifica o texto em Base64
        b64 = base64.b64encode(texto.encode("utf-8")).decode("utf-8")
        # Cria um link com o atributo download e formata com um botão
        html = (
            f'<a download="{nome_arquivo}" '
            f'href="data:text/plain;base64,{b64}" target="_blank">'
            f'<button style="padding:10px; font-size:16px; margin:5px;">'
            f'Clique aqui para baixar o arquivo {nome_arquivo}'
            f'</button></a>'
        )
        return HTML(html)

    # Cria o botão de download para o arquivo atual
    botao = criar_botao_download(texto_extraido, nome_txt)
    botoes_download.append(botao)

    # Exibe também o texto extraído (opcional)
    print(f"Texto extraído de {filename}:")
    print(texto_extraido)
    print("\n" + "="*80 + "\n")

# Exibe todos os botões de download
display(VBox(botoes_download))


## DOWNLOAD DO ARQUIVO TXT
o usuário poderá escolher a pasta onde deseja salvar o arquivo .txt por meio de um seletor de diretórios. Após clicar no botão, o arquivo será salvo e um link para download será exibido.

In [ ]:
def criar_link_download(texto, nome_arquivo):
    """
    Cria um link HTML que permite o download do conteúdo passado como string.
    """
    # Codifica o texto em Base64
    b64 = base64.b64encode(texto.encode()).decode()
    # Cria um link com o atributo download
    html = (
        f'<a download="{nome_arquivo}" '
        f'href="data:text/plain;base64,{b64}" target="_blank">'
        f'Clique aqui para baixar o arquivo {nome_arquivo}</a>'
    )
    return HTML(html)

# Exibe o link para download
display(criar_link_download(texto_extraido, nome_txt))
